[PC8TPMCP](https://moodle.bordeaux-inp.fr/course/view.php?id=4&section=5) Travaux pratiques Modélisation et Conduite des Procédés $\bullet$ ENSCBP - Bordeaux INP $\bullet$ [Nicolas Régnier](mailto:nicolas.regnier@enscbp.fr)

***

**À lire en premier**

Vous venez d'ouvrir un "Notebook Jupyter" ! Si c'est la première fois $\rightarrow$ consultez le [Tutoriel Jupyter](_tutoriel-jupyter.ipynb).
<br>Si vous avez besoin de réviser les bases du langage Matlab $\rightarrow$ consultez le [Tutoriel Matlab / Octave](_tutoriel-matlab.ipynb).
<br>Les exercices qui suivent font appel à des `fonctions Matlab`. Chaque fois que vous avez besoin d'aide $\rightarrow$ consultez le [Guide des fonctions Matlab](_guide-fonctions.ipynb).

***

***

**PC8TPMCP : TRAVAUX PRATIQUES MODÉLISATION ET CONDUITE DE PROCÉDÉS**

**NOM ET PRÉNOM** - ENSCBP CGP 2A - 2019/2020

***

# Introduction

Ces travaux pratiques sont une application des techniques de modélisation et de commande des procédés. Ils ont pour support des procédés simulés numériquement, dont le comportement est proche des systèmes expérimentaux étudiés lors des Travaux Pratiques Instrumentation et Traitement du Signal du premier semestre (PC7TPITS). Tous les traitements numériques sont réalisés dans ce Notebook avec le langage Matlab / Octave.

Cette étude comporte deux parties principales : l’une de modélisation au cours de laquelle sera défini un modèle ARX du procédé à partir de données expérimentales (simulées), et l’autre de conduite qui consistera à développer un régulateur RST à partir du modèle, à le régler et à le mettre en œuvre.

> **CONSIGNES IMPORTANTES**
>
> Toutes les cellules qui ont l'apparence de celle-ci, c'est-à-dire dont le texte est décalé vers la droite et précédé d'une barre grise verticale, contiennent des consignes destinées à vous guider dans votre travail. Vous pourrez les supprimer avant de rendre votre devoir (ceci est rappelé dans le paragraphe final "Finalisation et remise de votre travail"). Des consignes complémentaires sont inscrites dans les cellules de code, en tant que commentaires.
>
> Le reste du texte de ce Notebook introduit et décrit le travail réalisé, et reprend l'essentiel du cours MCPRO. Vous n'avez pas à le modifier, mais vous pouvez le faire si vous le jugez nécessaire ou utile.
>
> L'essentiel de votre travail doit avoir lieu dans les cellules de code, que vous devez compléter et exécuter pour obtenir vos résultats. Ainsi, vous n'avez quasiment pas de travail de rédaction à réaliser. Cependant :
>
> - Si vous le souhaitez *vous pouvez* ajouter des cellules de texte (Markdown) dans le Notebook, ou modifier le texte existant, par exemple pour analyser et commenter certains de vos résultats. À vous de juger, il n'y a aucune obligation mais ce n'est pas interdit.
> - *Vous devez* modifier deux cellules de texte particulières : celle dans laquelle vous devez écrire vos nom et prénom au début du Notebook (faites-le tout de suite avant d'oublier !), et le paragraphe "Conclusion" à la fin du Notebook.
>
> Vous pouvez réaliser ce travail en plusieurs fois. N'oubliez pas d'enregistrer régulièrement votre Notebook.

## Sommaire

1. [Introduction](#Introduction)
2. [Présentation du procédé](#Présentation-du-procédé)
3. [Modélisation du procédé](#Modélisation-du-procédé)
    1. [Préparation de l'expérience d'identification](#Préparation-de-l'expérience-d'identification)
    2. [Réalisation de l'expérience d'identification](#Réalisation-de-l'expérience-d'identification)
    3. [Prétraitement des mesures pour l'identification](#Prétraitement-des-mesures-pour-l'identification)
    4. [Recherche de la structure globale du modèle](#Recherche-de-la-structure-globale-du-modèle)
    5. [Vérification des incertitudes des coefficients estimés](#Vérification-des-incertitudes-des-coefficients-estimés)
    6. [Validation du modèle sur les données d'identification](#Validation-du-modèle-sur-les-données-d'identification)
    7. [Validation du modèle sur d'autres données](#Validation-du-modèle-sur-d'autres-données)
4. [Conduite du procédé](#Conduite-du-procédé)
    1. [Synthèse du régulateur](#Synthèse-du-régulateur)
    2. [Mise en œuvre du régulateur](#Mise-en-œuvre-du-régulateur)
    3. [Vérification des performances du régulateur](#Vérification-des-performances-du-régulateur)
5. [Conclusion](#Conclusion)
6. [Finalisation et remise de votre travail](#Finalisation-et-remise-de-votre-travail)

# Présentation du procédé

Le procédé étudié est un aérotherme électrique. Il s'agit d'un appareil permettant de produire un flux d'air à température constante, souvent utilisé pour le chauffage de locaux de grands volumes comme des usines, entrepôts, grandes surfaces commerciales, etc.

Un aérotherme électrique est constitué d'un ventilateur qui met en mouvement un flux d'air, d'une résistance électrique (l'actionneur) qui chauffe plus ou moins le flux d'air traversant l'appareil, et d'un capteur de température du flux d'air chauffé (voir figure 1).

![Image](images/aerotherme.png)

*Figure 1 : schéma de principe d'un aérotherme électrique*

L'objectif final de cette étude est de définir un régulateur de la température du flux d'air chauffé.

L'unité de la variable d'entrée du procédé (puissance appliquée à la résistance électrique de chauffage) est le pourcent $\%$ (il s'agit du pourcentage de la puissance maximum), et l'unité de la variable de sortie du procédé (capteur de température) est le degré Celsius $^{\circ} C$. Le procédé est schématisé sous forme d'un système à une entrée $u$, la puissance de chauffe, et une sortie $y$, la température (voir figure 2).

![Image](images/procede.png)

*Figure 2 : représentation schématique du procédé*

L'appareil a déjà été étudié et certaines données concernant le procédé et le système de mesure ont déjà été choisies ou estimées :

- La période d'échantillonnage du système de mesure, $Te$, choisie d'après l'analyse fréquentielle de la réponse du procédé à un signal d'excitation.
- Le temps de réponse à 98% du procédé, $T_{98\%}$, estimé d'après la réponse indicielle du procédé.
- Le temps de réponse à 95% du procédé, $T_{95\%}$, estimé d'après la réponse indicielle du procédé.

> Pour obtenir ces données pour votre propre procédé, exécutez la cellule de code ci-dessous (bouton `Run`▸ en haut de cette page, ou `maj`+`entrée` au clavier).

In [ ]:
% Caractéristiques temporelles du système de mesure et du procédé

[Te,T98,T95] = process()

La période d'échantillonnage $Te$ et les temps de réponse $T_{98\%}$ et $T_{95\%}$ à 98% et 95% sont respectivement affectés aux variables `Te`, `T98`, et , `T95`. Leurs valeurs sont données en secondes.

> Note : ces valeurs vous sont propres, vous pourrez toujours les retrouver de cette façon, et elles sont différentes d'un utilisateur du simulateur à un autre.

Pour obtenir la réponse du procédé à un signal d'entrée quelconque, il suffit de le passer en argument d'entrée à la fonction `process`. Celle-ci renvoie alors le signal de sortie simulé en réponse au signal d'entrée, et le temps correspondant. Le signal d'entrée doit être un vecteur de valeurs qui sont appliquées au procédé selon sa période d'échantillonnage $Te$.

Par exemple, pour obtenir des réponses indicielles du procédé, il suffit de définir un signal d'entrée `u` sous forme d'échelons de durée supérieure au temps de réponse à 98%.

> Exécutez la cellule de code ci-dessous pour obtenir quelques réponses indicielles du procédé.
>
> Il s'agit seulement d'une démonstration non-nécessaire pour la suite (puisque les temps de réponse ont déjà été estimés), mais **vous pourrez vous inspirer de ces instructions pour la suite de l'étude**.

In [ ]:
% Durée d'un échelon : 2 fois le temps de réponse à 98% (choix arbitraire permettant une stabilisation correcte)
% Nombre de valeurs échantillonnées correspondant à cette durée : durée / période d'échantillonnage

n = round(2*T98/Te);

% Définition du signal d'entrée : un échelon avec u=50% (pour la mise en régime) 
% + un échelon avec u=70% (soit +20% par rapport au point de fonctionnement)
% + un échelon avec u=50% (retour au point de fonctionnement)
% + un échelon avec u=30% (soit -20% par rapport au point de fonctionnement)

u = [ones(n,1)*50 ; ones(n,1)*70 ; ones(n,1)*50 ; ones(n,1)*30];

% Calcul de la réponse du procédé

[y,t] = process(u);

% Tracé des signaux d'entrée et de sortie

subplot(2,1,1)
stairs(t,u)
ylim([0 100])
ylabel('Pourcentage de chauffe')
title('Exemple de reponses indicielles')

subplot(2,1,2)
plot(t,y)
ylabel('Temperature (\degC)')
xlabel('Temps (s)')

Observations :

1. La valeur initiale de la température en sortie du procédé est la température ambiante.
2. Les mesures de température sont entachées d'un bruit de mesure résiduel.

# Modélisation du procédé

Références : [MCPRO5 - identification paramétrique des fonctions de transfert discrètes](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22588) et [Fiche-résumé : modélisation linéaire ARX](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=17913)

L'identification paramétrique consiste à déterminer la structure et les coefficients d'un modèle qui permet de représenter au mieux des séries de mesures réalisées en sortie d'un procédé. Pour que ces données soient riches en informations, et permettent de déterminer un modèle apte à reproduire le comportement du procédé dans diverses situations, les mesures utilisées pour la modélisation sont obtenues lors d'expériences particulières qui consistent à appliquer un signal d'excitation adéquat à l'entrée du procédé.

Le type de modèle recherché est un modèle dynamique discret de type ARX (Auto Regressive eXogenous model) dont la forme générale est la suivante (équation aux différences) :

$A \left( q^{-1} \right) \cdot y(k) = q^{-r} \cdot B \left( q^{-1} \right) \cdot u(k)$

avec : $A \left( q^1 \right) = 1 - a_1 \cdot q^{-1} - a_2 \cdot q^{-2} \ldots - a_{n_a} \cdot q^{-n_a}$ et $B \left( q^1 \right) = 0 + b_1 \cdot q^{-1} + b_2 \cdot q^{-2} \ldots + b_{n_b} \cdot q^{-n_b}$

$n_a$ et $n_b$ sont les ordres du modèle, $r$ est son retard, les $a_i$ et $b_i$ sont ses coefficients. La modélisation consiste à déterminer les valeurs optimales de tous ces éléments.

## Préparation de l'expérience d'identification

L'expérience d'identification consiste à appliquer un signal d'excitation adéquat à l'actionneur (l'entrée) du procédé, afin d'obtenir des signaux de mesures entrée/sortie riches en informations. Il s'agit ici d'une séquence binaire pseudo aléatoire (SBPA) dont les caractéristiques sont choisies d'après les données disponibles et les contraintes imposées.

Choix du type de signal et de son amplitude :

- Type : SBPA
- Point de fonctionnement : $u_0 = 50 \%$
- Amplitude de variation autour du point de fonctionnement : $\Delta u = \beta = 20 \%$

Choix des paramètres de la SBPA ($n$ et $k$) : 

- Critère basse fréquence : $n \cdot k \ge \dfrac{T_{98\%}}{Te}$  (lié à la durée du plus long échelon de la séquence)
- Critère haute fréquence : $k \le 3$  (lié à la borne haute de la bande passante du signal)
- Critère nombre de mesures : $\left( 2^n - 1 \right) \cdot k \ge 200$  (valeur permettant une identification correcte, en pratique)
- Critère temps : $\left( 2^n - 1 \right) \cdot k \cdot Te \le 14400$  (en secondes, lié au temps disponible supposé ici de 4h)

> En utilisant vos valeurs de $Te$ et de $T_{98\%}$, choisissez un couple ($n$, $k$) adéquat pour le signal d'excitation de l'expérience d'identification.
>
> Commencez par exemple  par déterminer la plus petite valeur de $n$ qui permet de satisfaire le premier critère quand $k$ a sa valeur maximum ; vérifiez si dans ce cas les autres critères sont satisfaits, et si ce n'est pas le cas réajustez $n$ et $k$ à partir de ces valeurs.
>
> Utilisez et complétez la cellule de code suivante et exécutez-la (vous pouvez l'utiliser pour tester divers couples ($n$, $k$), et conserver uniquement votre choix final pour le compte-rendu) .

In [ ]:
% À compléter avec vos calculs et vos choix (remplacez les points d'interrogation)

% le produit n*k doit être supérieur à 

nk_min = ?

% Valeurs choisies de n et de k

n = ?
k = ?

% Nombre de mesures N et durée de l'expérience Texp

N = ?
Texp = ? % en secondes

Génération du signal d'excitation à l'aide de la fonction `sbpa` :

> Consultez le mode d'emploi de la fonction `sbpa` dans le [Guide des fonctions Matlab](_guide-fonctions.ipynb), ou en tapant `help sbpa`, pour générer un signal SPBA correspondant aux valeurs de $n$, $k$, $u_0$, et $\beta$ choisies précédemment.

In [ ]:
% À compléter par vos paramètres (remplacez le point d'interrogation)

% Génération du signal d'excitation (SBPA) pour l'expérience d'identification

u = sbpa(?);

## Réalisation de l'expérience d'identification

La réalisation de l'expérience d'identification consiste à appliquer le signal d'excitation généré (SBPA) à l'entrée du procédé et à enregistrer sa réponse. Étant réalisée en simulation, cette étape prendra un temps négligeable. Mais expérimentalement cette expérience aurait pris plusieurs heures (voir durée `Texp` calculée précédemment).

> Appliquez le signal d'excitation `u` que vous avez généré à l'entrée du procédé à l'aide de la fonction `process`, recueillez la réponse et tracez les signaux.
>
> Vous pouvez vous inspirer du code utilisé précédemment pour calculer et tracer des réponses indicielles, et/ou consulter l'aide et le mode d'emploi de la fonction `process`.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Calcul de la réponse du procédé

?

% Tracé des signaux d'entrée et de sortie

?

## Prétraitement des mesures pour l'identification

Le prétraitement des données de l'expérience d'identification consiste à :

1. Supprimer les premières valeurs des signaux, c'est-à-dire celles qui correspondent à la mise en régime et qui ont été enregistrée avant stabilisation du procédé.
2. Retirer les valeurs moyennes des signaux, c'est-à-dire leurs composantes continues.

La raison de ces deux traitements de données est que le type de modèle choisi (ARX) ne peut représenter que des variations autour d'un point de fonctionnement.

> Le prétraitement est décrit et illustré dans le cours [MCPRO5 - identification paramétrique des fonctions de transfert discrètes](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22588). Réalisez les deux traitements de la façon suivante : 
>
> 1. Tronquez les trois vecteurs `t`, `u`, et `y` de façon à ne retenir que les valeurs à partir de l'indice `n*k+1` jusqu'à la fin, c'est-à-dire toutes les valeurs au-delà du nombre de points $n \cdot k$ qui correspond par définition à un temps supérieur au temps de réponse à 98% (donc approximativement à la durée de la mise en régime).
> 2. Puis retirez aux vecteurs `u` et `y` leur valeurs moyennes calculées à l'aide de la fonction `mean` (il est inutile d'essayer de supprimer une tendance linéaire à l'aide de la fonction `detrend`, car il n'y en a pas dans ces données simulées).
>
> Tracez les signaux prétraités.

In [ ]:
% À compléter (remplacez les points d'interrogation)
% (vous pouvez changer les noms des variables si vous le souhaitez, mais dans ce cas tenez-en compte dans les calculs suivants !)

% Troncation des vecteurs (y compris le temps) pour supprimer la partie non-stabilisée

t = ?;
u = ?;
y = ?;

% Suppression de la composante continue des signaux

u = ?;
y = ?;

% Tracé des signaux d'entrée et de sortie prétraités

?

## Recherche de la structure globale du modèle

La recherche de la structure globale consiste à choisir les ordres $n_a=n_b$ et le retard $r$ du modèle, de façon à ce qu'il représente au mieux les données (prétraitées) de l'expérience d'identification, tout en restant aussi simple que possible. C'est ce qu'on appelle le principe de parcimonie : il n'est pas judicieux de choisir un modèle d'ordre élevé (donc ayant beaucoup de coefficients) si un modèle plus simple (d'ordre plus faible) suffit
à atteindre quasiment la même qualité de modélisation. Cette qualité est évaluée par la variance des résidus $\sigma^2_{\hat \varepsilon}$, les résidus $\hat\varepsilon(k) = y(k) - \hat y(k)$ étant la différence entre les mesures (prétraitées) $y(k)$ et les prédictions du modèle $\hat y(k)$.

La procédure générale est la suivante : 

1. Calculer la variance des résidus pour des modèles d'ordre croissant en commençant par le plus simple (c'est-à-dire d'ordre 1), et pour différents retards en commençant à 0 (à moins de disposer d'informations supplémentaires).
2. Sélectionner l'ordre du modèle selon un compromis : ordre le plus petit possible, au-delà duquel la diminution de la variance des résidus n'est plus significative en passant à l'ordre supérieur.
3. Sélectionner le retard pour lequel la variance des résidus est la plus petite pour l'ordre sélectionné.

Ce choix étant partiellement subjectif, il est préférable de le réaliser en analysant des graphiques de la variance des résidus en fonction de l'ordre ou du retard du modèle, plutôt que simplement à partir de tableaux de valeurs numériques. Il est possible que cette procédure conduise à retenir plusieurs modèles candidats : dans ce cas les vérifications suivantes, appliquées à chacun des candidats, permettront de faire un choix définitif.

> Déterminez la structure d'un modèle ARX adéquat selon cette procédure, qui est détaillée et illustrée dans le cours [MCPRO5 - identification paramétrique des fonctions de transfert discrètes](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22588). Pour cela, utilisez la fonction `identification` qui reçoit vos données prétraitées et les structures des modèles à tester, et renvoie les variances des résidus correspondantes. Cette fonction peut être utilisée de différentes façon (consultez sa description et inspirez-vous des exemples) : utilisez la version qui permet de calculer en un seul appel différentes valeurs des ordres $n_a = n_b$ pour différents retards $r$.
>
> En pratique, calculez la variance des résidus pour un ordre global du modèle $n_a = n_b = n$ variant de 1 à 5 et pour un retard $r$ variant de 0 à 2. 
>
> Pour vous aider à choisir une structure de modèle, vous pouvez tracer différents graphiques : variance en fonction de l'ordre (ce qui donnera 3 courbes, une pour chaque retard), variance en fonction du retard (ce qui donnera 5 courbes, une pour chaque ordre). Si nécessaire vous pouvez utiliser la fonction `ylim` pour changer les limites de l'axe des ordonnées du graphique, afin de "zoomer" sur les faibles valeurs. Vous pouvez aussi essayer de faire varier l'ordre $n_a = n_b = n$ de 0 à 5 : l'ordre 0 est fictif, il correspond à l'absence de modèle ($\hat y(k)=0, \forall k$), et la variance correspondante est donc celle de $y(k)$. Cette valeur n'est pas indispensable, mais peut donner par comparaison une échelle de lecture intéressante de l'évolution de la variance.
>
> À l'aide de ces graphiques, sélectionnez selon la procédure proposée un modèle dont la structure $[n_a n_b r]$ vous semble appropriée. Vous préciserez ce choix dans le paragraphe suivant.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Calcul des variances des résidus pour n=1:5 (ou 0:5) et r=0:2

vareps = identification(?);

% Tracés des variances des résidus en fonction de l'ordre du modèle et/ou de son retard

?

## Vérification des incertitudes des coefficients estimés

La méthode d'identification peut fournir les coefficients du modèle, c'est-à-dire les valeurs $a_i$ et $b_i$ des polynômes $A \left( q^{-1} \right)$ et $B \left( q^{-1} \right)$ pour une structure $[na, nb, r]$ donnée, ainsi que les incertitudes associées $ic(a_i)$ et $ic(b_i)$. Si certaines incertitudes sont non-négligeables par rapport aux valeurs absolues des coefficients, le modèle ne peut pas être retenu en l'état : le ou les coefficients en question doivent être supprimés du modèle, ce qui revient en général à réduire $na$ et/ou $nb$.

> À l'aide de la fonction `identification` (utilisée sous une forme différente, voir description), affichez les coefficients du modèle retenu d'après l'analyse de la variance des résidus, et leurs intervalles de confiance qui sont assimilés à des incertitudes.
>
> Vérifiez que les intervalles de confiance sont négligeables par rapport aux valeurs absolues des coefficients estimés. Si ce n'est pas le cas, essayez une autre structure de modèle, par exemple en réduisant $na$ et/ou $nb$ (qui peuvent donc être différents à ce stade) et en faisant à nouveau appel à la fonction `identification`.
>
> Les intervalles de confiance pour les premiers coefficients de $A \left( q^{-1} \right)$ et de $B \left( q^{-1} \right)$ (qui valent respectivement  1 et 0) sont nuls : cela signifie que ces valeurs sont "certaine" et n'ont bien sûr pas à être remises en cause.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Modèle retenu d'après l'analyse de la courbe de variance

na = ?
nb = ?
r = ?
[A,icA,B,icB] = identification(?)

## Validation du modèle sur les données d'identification

À ce stade, la qualité du modèle a été évaluée uniquement d'après la variance des résidus $\sigma^2_{\hat \varepsilon}$ calculées à partir des données d'identification. C'est un indicateur pratique (scalaire positif) pour la recherche de la structure et des paramètres optimaux du modèle. 

Il est généralement utile de **comparer graphiquement les mesures $y(k)$ et les valeurs prédites par le modèle $\hat y(k)$**, afin de vérifier l'accord entre les deux 

> Calculez $\hat y(k)$ grâce à la fonction `prediction` pour le modèle que vous avez retenu (défini par `A`, `B`, et `r`)
>
> Tracez $y(k)$ et $\hat y(k)$ sur un même graphique afin de les comparer.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Calcul de la sortie prédite par le modèle

yp = prediction(?);

% Tracé et comparaison de la sortie prédite par le modèle avec les valeurs mesurées

?

Il est souvent difficile de distinguer les deux courbes $y(k)$ et $\hat y(k)$, ce qui ne prouve pas forcément l'absence de problèmes ni l'impossibilité d'améliorer encore le résultat.

Pour obtenir une indication plus précise, il est plus utile de **représenter graphiquement les résidus $\hat\varepsilon(k)$**, ce qui *peut* permettre de repérer rapidement des défauts dans le prétraitement des données ou la modélisation : en effet, $\hat\varepsilon(k)$ doit être un signal aléatoire centré sur zéro, car il est la différence entre les valeurs $\hat y(k)$ prédites par le modèle et les mesures prétraitées $y(k)$. Donc si le modèle est adéquat les résidus doivent être égaux aux bruits de mesure, c'est-à-dire aléatoires.

> Calculez les résidus, qui sont les différences entre $\hat y(k)$ et $y(k)$.
>
> Tracez-les en fonction du temps.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Calcul des résidus

epsilon = ?;

% Tracé des résidus en fonction du temps

?

Pour vérifier plus rigoureusement leur caractère aléatoire, il suffit de **calculer et tracer la fonction d'autocorrélation des résidus $\hat\varepsilon(k)$** et de s'assurer qu'elle est proche de zéro. En pratique, on vérifiera qu'elle est comprise dans un certain intervalle de confiance autour de zéro, par exemple $\displaystyle \pm \, 2 \cdot \frac{\sigma^2_{\hat \varepsilon}}{\sqrt{N}}$ pour un intervalle de confiance à 97.5% ($N$ est la longueur du vecteur de mesures prétraitées).

> Calculez la fonction d'autocorrélation des résidus à l'aide de la fonction `xcov` en utilisant la méthode de normalisation 'unbiased' (voir description de la fonction).
>
> Calculez les intervalles de confiances d'après la formule ci-dessus (en utilisant les fonctions `var` et `length` respectivement pour la variance et la longueur du vecteur de mesures).
>
> Tracez sur un même graphique la fonction d'autocorrélation et les intervalles de confiance sous forme de deux droites horizontales de part et d'autre de l'axe $y=0$. Vous pouvez consulter le cours [MCPRO5 - identification paramétrique des fonctions de transfert discrètes](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22588) pour voir un aperçu de ce que vous devez obtenir.
>
> Il est normal que les valeurs de la fonction d'autocorrélation divergent aux extrémités, en raison de la méthode de normalisation. Il ne faut donc pas en tenir compte.

In [ ]:
% À compléter (remplacez les points d'interrogation)

% Calcul de la fonction d'autocorrélation des résidus et des intervalles de confiance à 97.5%

[c,lags] = xcov(?);
ic = ?;

% Tracé de la fonction d'autocorrélation et des intervalles de confiance

?

Le résidu d'estimation $\hat\varepsilon(k)$, ou sa variance, ou sa fonction d'autocorrélation, ne peuvent pas être facilement comparés d'une expérience à une autre (dont la durée et les variations peuvent être très différentes), et encore moins d'un procédé à un autre (pour les mêmes raisons, et parce qu'en plus l'unité des mesures peut être différente). Pour permettre cette comparaison, un critère de qualité "normalisé" peut être utilisé. Par exemple il est courant de **calculer le coefficient de détermination $R^2$**, qui rapporte la variance du résidu à celle des mesures $\sigma^2_y$ et donne un scalaire sans unité. Il s'exprime de la façon suivante : $R^2 = \displaystyle \frac{\sigma^2_y - \sigma^2_{\hat \varepsilon}}{\sigma^2_y}$.

> Calculez la valeur du coefficient de détermination $R^2$ à l'aide de la fonction `var`.

In [ ]:
% À compléter (remplacez le point d'interrogation)

% Calcul du coefficient de détermination de la modélisation

R2 = ?

## Validation du modèle sur d'autres données

En toute rigueur, le modèle déterminé précédemment n'est encore à ce stade qu'un "bon candidat". Il permet de reproduire correctement les mesures utilisées pour son identification, mais rien ne prouve encore qu'il soit capable de reproduire correctement d'autres mesures obtenues sur le même procédé dans des circonstances différentes. L'étape qui consiste à s'en assurer est la véritable étape de validation du modèle, réalisée à partir d'un autre jeu de mesures et donc d'une autre expérience. 

Réaliser cette validation nécessite donc :

1. La préparation d'une autre expérience, dite de validation. 
2. La réalisation de cette expérience de validation.
3. Le prétraitement des mesures obtenues.
4. La validation proprement dite du modèle *précédemment obtenu* sur les données de validation.

**Il ne faut surtout pas identifier un nouveau modèle** sur ces nouvelles données, mais bien tester sur elles le modèle que vous avez déjà sélectionné.

La nouvelle expérience de validation sera définie par un signal d'excitation différent de celui de l'expérience d'identification : pour simplifier la procédure il sera choisi de même nature (SBPA), mais avec des paramètres différents. On prendra par exemple :

- $n_{validation} = n_{identification} - 1$   (ce qui réduit la durée de l'expérience)
- $k_{validation} = max \left( k_{identification} + 1, 3 \right)$   (la valeur de $k$ est donc inchangée si elle était déjà égale à $3$)
- $\Delta u = \beta = 25 \%$  (au lieu de $20 \%$ pour l'indentification)

Il est possible que les nouvelles valeurs de $n$ et $k$, choisies de cette façon arbitraire, ne remplissent pas les critères utilisés pour générer la SPBA d'identification. Mais ce n'est pas grave dans ce cas, car cette expérience de validation ne servira pas à identifier un nouveau modèle du procédé.


> Réalisez les quatre étapes de la validation. 
>
> Vous pouvez procéder de façon très simple et rapide en copiant les cellules de code adéquates depuis les chapitres précédents, et en les collant ci-dessous. Vous n'aurez ainsi qu'à modifier les valeurs qui définissent le nouveau signal d'excitation SBPA, et à exécuter ces cellules collées.

In [ ]:
% À vous de jouer !

# Conduite du procédé

Références : [MCPRO6 - Synthèse des régulateurs RST](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22589) et [Fiche-résumé : régulation RST](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=18306)

La synthèse d'un régulateur consiste à déterminer la structure et les coefficients des polynômes (ou fonctions de transfert) qui le définissent, soit dans le cas d'un régulateur RST : $R(q^{-1})$, $S(q^{-1})$,  et $T(q^{-1})$. Ces éléments seront déterminés par identification polynomiale, en écrivant d'une part l'expression générale du système en boucle fermée (dans laquelle interviennent les polynômes du régulateur à déterminer, et la fonction de transfert connue du modèle du procédé), et d'autre part une fonction de transfert de référence qui décrit le comportement dynamique que l'on souhaite obtenir pour le système en boucle fermée.

La connaissance d'un modèle du procédé, tel que celui déterminé précédemment, est donc indispensable. 

La mise en œuvre d'un régulateur consiste à appliquer le régulateur au procédé réel (ou ici, au procédé simulé) en boucle fermée, à le solliciter avec diverses valeurs de consigne et à vérifier que le système réagit correctement (comportement dynamique transitoire, respect de la consigne malgré bruits de mesure et perturbations).

## Synthèse du régulateur

Le schéma d'un système en boucle fermée, constitué d'un régulateur RST et d'un modèle de procédé ARX, est représenté sur la figure 3.

![Image](images/regulateur.png)

*Figure 3 : schéma du système en boucle fermée*

La fonction de transfert de ce système en boucle fermée est la suivante :

$y(k) = \dfrac{q^{-r} \cdot B(q^{-1}) \cdot T(q^{-1})}{A(q^{-1}) \cdot R(q^{-1}) + q^{-r} \cdot B(q^{-1}) \cdot S(q^{-1})} \cdot w(k)$

C'est le dénominateur de cette fonction de transfert qui est identifié à celui d'une fonction de référence, car il définit le comportement dynamique du système en boucle fermée. Il existe de nombreux choix possibles, celui qui est fait ici est le suivant : la fonction de transfert de référence est un système dynamique continu d'ordre 2, dont le comportement est défini par sa pulsation naturelle $\omega_0$ et son facteur d'amortissement $\zeta$. Cette fonction de transfert de référence doit être discrétisée (car tout le système est échantillonné) et son dénominateur est noté $D(q^{-1})$.

L'identité polynomiale à résoudre est donc finalement : $A(q^{-1}) \cdot R(q^{-1}) + q^{-r} \cdot B(q^{-1}) \cdot S(q^{-1}) = D(q^{-1})$.

Le polynôme de référence est de la forme : $D(q^{-1}) = 1 - d_1 \cdot q^{-1} - d_2 \cdot q^{-2}$, avec (résultat de la discrétisation d'une fonction de transfert continue d'ordre deux) :

$d_1 = 2 \cdot \displaystyle e^{- \zeta \cdot \omega_0 \cdot Te} \cdot cos \left( \omega_0 \cdot Te \cdot \sqrt{1 - \zeta^2}  \right)$

$d_2 = - \displaystyle e^{- 2 \cdot \zeta \cdot \omega_0 \cdot Te}$

Les coefficients dépendent donc des paramètres "de réglage" $\omega_0$ et $\zeta$. Ils peuvent être choisis dans une large plage, mais les recommandations suivantes peuvent être utilisées pour faire un choix initial (éventuellement réajusté après observation des résultats) : 

$0.7 \le \zeta \le 1$  : amortissement entre critique et total.

$\dfrac{0.25}{Te} \le \omega_0 \le \dfrac{1.5}{Te}$  : dynamique cohérente avec celle du procédé en boucle ouverte (représentée par la période d'échantillonnage $Te$).

Pour que le système soit soluble, c'est-à-dire pour qu'il y ait autant d'équations que d'inconnues, les ordres des polynômes $R(q^{-1})$ et $S(q^{-1})$ doivent être soigneusement choisis. Il suffit de prendre : $n'_r = n_s = max(n_a+1,n_b+r)-1$ et les structures suivantes :

$R(q^{-1}) = \left( 1-q^{-1} \right) \cdot \left( 1 - r_1 \cdot q^{-1} - r_2 \cdot q^{-2} - \ldots - r_{n'_r} \cdot q^{n'_r} \right)$

$S(q^{-1}) = s_0 + s_1 \cdot q^{-1} + s_2 \cdot q^{-2} + \ldots + s_{n_s} \cdot q^{n_s}$

On peut noter que le polynôme $R(q^{-1})$ contient un terme intégrateur $(1-q^{-1})$, et que son ordre est $n_r = n'_r + 1$.

Le nombre d'inconnues (et donc d'équations) de ce système est égal à $2 \cdot n_s + 1$. 

**Choix des paramètres $\omega_0$ et $\zeta$ et calcul du polynôme de référence $D(q^{-1})$** :

> Faites un premier choix arbitraire pour les valeurs de `omega0` et `zeta` (dans les intervalles indiqués), vous pourrez les modifier plus tard. 
>
> Calculez `d1` et `d2` à partir de ces valeurs et des formules fournies. 

In [ ]:
% À compléter (remplacez les points d'interrogation)
% Choix des valeurs de la pulsation naturelle et du facteur d'amortissement

omega0 = ?
zeta = ?

% Calcul des coefficients de la fonction de transfert de référence

d1 = ?
d2 = ?
D = [1 -d1 -d2];

**Détermination des ordres des polynômes $R(q^{-1})$ et $S(q^{-1})$ du régulateur** :

> Déterminez l'ordre `ns` des polynômes $S(q^{-1})$ et $R'(q^{-1})$ (ce dernier n'étant qu'une notation pour la partie de $R(q^{-1})$ qui est facteur du terme intégrateur $(1-q^{-1})$).

In [ ]:
% À compléter (remplacez le point d'interrogation)
% Ordres des polynômes R' et S

ns = ?

**Résolution du système d'équations** :

> Résolvez le système d'équations correspondant à l'identité polynomiale $A(q^{-1}) \cdot R(q^{-1}) + q^{-r} \cdot B(q^{-1}) \cdot S(q^{-1}) = D(q^{-1})$. Pour cela vous avez deux possibilités, et dans tous les cas vous pouvez vous inspirer de l'exercice 2.3 des [Travaux Dirigés Modélisation et Conduite de Procédés](https://moodle.bordeaux-inp.fr/course/view.php?id=4&section=4) qui démontre la résolution d'un système à 3 équations ( $n'_r = n_s = 1$).
>
> 1. Soit écrire et résoudre le système "à la main", ce qui est facile lorsque le nombre d'équations est limité (comme dans l'exemple des TD MCPRO), mais devient de plus en plus complexe lorsqu'il augmente. Si vous choisissez cette méthode, écrivez simplement le résultat dans une cellule de code (quelque chose comme `R = [?]; S = [?];`). Supprimez le contenu de la cellule de code suivante (ou remplacez son contenu par vos résultats), car elle ne vous sert à rien et risque de générer des erreurs d'exécution.
> 2. Soit utiliser les fonctionnalités de calcul symbolique de Matlab / Octave pour résoudre le système, quel que soit son ordre. Vous pouvez **adapter** la cellule de code suivante en fonction de vos ordres et retard. Si vous choisissez cette méthode, lisez bien les consignes en commentaires et prenez votre temps ! Ce genre de calcul nécessite une syntaxe très rigoureuse, le moindre écart générant des erreurs qu'il est souvent difficile d'interpréter. Enfin soyez patient(e), le calcul peut prendre plusieurs secondes.

In [ ]:
% Exemple de résolution pour na = 3, nb = 3, r = 1, À ADAPTER !
%
% Pour ne pas écraser les précédentes, de nouvelles variables sont définies pour les versions symboliques
% des polynômes A, B, R, S, et D, en ajoutant un 's' à leur noms --> As, Bs,Rs, Ss, Ds
%
% Ne modifiez pas cette ligne
warning('off');

% Ne modifiez pas cette ligne
%
% Le symbole q représente en réalité q^-1. Ainsi q^2 représente q^-2, etc.
syms q

% Écrivez les polynômes As et Bs (le 's' est donc pour 'symbolique') conformément à votre modèle (ajoutez ou supprimez des termes)
%
% Définitions symboliques des polynômes du modèle de procédé 
As = A(1) + A(2)*q + A(3)*q^2 + A(4)*q^3;
Bs = B(1) + B(2)*q + B(3)*q^2 + B(4)*q^3;

% Sur la ligne commençant par 'syms', déclarez les coefficients ri et si correspondant à votre régulateur (ajoutez ou supprimez des termes)
% Sur les lignes suivantes, écrivez les polynômes Rs et Ss conformément à votre régulateur (ajoutez ou supprimez des termes)
%
% Définitions symboliques des polynômes du régulateur 
syms r1 r2 r3 s0 s1 s2 s3
Rs = (1 - q)*(1 - r1*q  - r2*q^2  - r3*q^3);
Ss = s0 + s1*q + s2*q^2 + s3*q^3;

% Ne modifiez pas cette ligne : l'ordre et la forme du polynôme Ds sont ici invariants
%
% Définition symbolique de la fonction de transfert de référence 
Ds = D(1) + D(2)*q + D(3)*q^2;

% Ne modifiez pas ces lignes
%
% Équation polynomiale à résoudre (P doit être nul)
P = As*Rs + q^r*Bs*Ss - Ds;
% Collecte des expressions par puissances de q --> système d'équations à résoudre (toutes doivent être nulles)
C = coeffs(P,q);

% Modifiez cette ligne : 
% - dans les crochets à gauche du signe égal, écrivez la liste de vos inconnues (ajoutez ou supprimez des termes)
% - à la fin de la parenthèse de la fonction solve, écrivez EXACTEMENT la même liste d'inconnues
% - au début de la parenthèse de la fonction solve, écrivez autant de termes C(i) qu'il y a d'équations
%   (et donc d'inconnues), en incrémentant i d'un terme à l'autre (ajoutez ou supprimez des termes)
%
% Résolution du système d'équations
[r1,r2,r3,s0,s1,s2,s3] = solve(C(1),C(2),C(3),C(4),C(5),C(6),C(7),r1,r2,r3,s0,s1,s2,s3);

% Modifiez ces lignes : adaptez la construction des vecteurs de coefficients pour les polynômes R et S,
% de façon à ce qu'ils correspondent aux ordres de ceux de votre régulateur (ajoutez ou supprimez des termes)
%
% Évaluation numériques des polynômes : la fonction 'conv' réalise un produit polynomial. Le vecteur 
% [1 -1] représente le polynôme 1 -q^-1. La fonction 'double' transforme les expressions en valeurs réelles.
R = conv([1 -1], double([1 -r1 -r2 -r3]))
S = double([s0 s1 s2 s3])

Il reste à déterminer le polynôme $T(q^{-1})$, qui n'intervient pas dans l'identité polynomiale. Il est facile de montrer que, pour que la fonction de transfert en boucle fermée ait un gain statique égal à un, il faut que $T(1) = \dfrac{D(1)}{B(1)}$. Cette égalité a une infinité de solutions. Nous choisissons la solution la plus simple dans laquelle $T(q^{-1})$ est d'ordre zéro, donc égal à un scalaire : $T(q^{-1}) = t_0 = \dfrac{D(1)}{B(1)}$

> Calculez `T` selon cette formule.
>
>Attention, l'expression mathématique $D(1)$ signifie "valeur du polynôme $D(q^{-1})$ pour $q=1$". Ce n'est pas du tout la même chose que l'expression informatique `D(1)` qui signifie "premier élément du vecteur `D`". Utilisez la fonction `sum` pour calculer les sommes des coefficients des polynômes `D` et `B`.

In [ ]:
% À compléter (remplacez le point d'interrogation)
% Calcul de T

T = ?

**Vérification des polynômes du régulateur :**

Avant de mettre en œuvre le régulateur, il est nécessaire de vérifier qu'il n'y a pas eu d'erreur lors de sa synthèse. Il faut :

1. Vérifier que $A(q^{-1}) \cdot R(q^{-1}) + q^{-r} \cdot B(q^{-1}) \cdot S(q^{-1}) = D(q^{-1})$.
2. Vérifier que $R(1) = 1$.
2. Vérifier que $T(1) = S(1)$.

Les deux dernières vérifications induisent que le gain statique de la fonction de transfert en boucle fermée est bien égal à un.

> Réalisez ces trois vérifications. 
>
> Pour la première, calculez le terme à gauche du signe égal en utlisant la fonction `conv` pour réaliser les produits de vos polynômes. Si le retard est non-nul, le terme $q^{-r}$ peut lui-même être représenté par un polynôme : `[0 1]` représente $q^{-1}$ soit un retard de 1, `[0 0 1]` représente $q^{-2}$ soit un retard de 2, etc. Il est possible que les deux termes de la somme n'aient pas la même taille, ce qui rendra la somme impossible : dans ce cas, ajoutez un ou plusieurs zéros *à la fin* du terme le plus court, de façon à ce qu'il ait la même taille que l'autre.
>
> Pour les deux dernières vérifications, utilisez la fonction `sum` pour calculer les sommes des coefficients des polynômes.
>
> Il est possible que certains termes qui devraient être nuls ne le soient pas complètement. Mais leurs ordres de grandeur doivent être nettement négligeables par rapport à ceux des autres, ce qui constituera une vérification suffisante.

In [ ]:
% À compléter (remplacez les points d'interrogation)
% Vérifications

?
?
?

## Mise en œuvre du régulateur

Le régulateur est mis en œuvre en boucle fermée de façon à contrôler le procédé. Dans cette configuration, l'opérateur de fournit plus un signal à appliquer à l'entrée du procédé (comme les signaux SBPA précédents) mais une consigne. C'est le régulateur qui définit les actions à appliquer à l'entrée $u$ du procédé, de façon à ce que sa sortie $y$ suive la consigne $w$ (voir figure 3).

Ces consignes doivent être physiquement réalisables, c'est-à-dire que la sortie du procédé doit pouvoir les atteindre. De plus, il est raisonnable que ces consignes soient *à l'intérieur* du domaine de variation de la sortie à partir duquel a été déterminé un modèle du procédé, qui a lui-même servi à définir le régulateur. En dehors de ce domaine, il n'est pas certain que la régulation puisse fonctionner correctement.

Pour un système réel, la consigne et son évolution sont définies par exemple selon un protocole opératoire. Pour tester le système, nous allons appliquer quelques créneaux de consigne de façon plus arbitraire, un peu à la façon du signal d'entrée qui a servi à la réalisation de réponses indicielles en boucle ouverte, *mais avec des unités et des amplitudes différentes*.

**Choix de la consigne  :**

> Déterminez d'abord les limites `wmin` et `wmax` de la consigne comme étant celles du domaine dans lequel a varié la sortie *non-prétraitée* du procédé lors de l'expérience d'identification. Vous pouvez prendre des valeurs approximatives lues sur les courbes. 
>
>Choisissez ensuite le nombre de valeurs `n` de chaque créneau de consigne : vous pouvez vous inspirer de l'exemple des réponses indicielles pour lequel ce nombre était égal à $2 \cdot \dfrac{T_{98\%}}{Te}$ (dans le but de laisser le temps au système de se stabiliser avant le créneau suivant).
>
>Finalement, construisez le vecteur des consignes `w`. Vous pouvez encore vous inspirer de l'exemple des réponses indicielles, en réalisant par exemple : un premier créneau dont la valeur est au milieu du domaine défini par `wmin` et `wmax`, un autre à `wmax`, un autre pour revenir au milieu du domaine, et un dernier à `wmin`.

In [ ]:
% À compléter (remplacez les points d'interrogation)
% Limites de variation de la consigne

wmin = ?
wmax = ?

% Nombre de valeurs de chaque créneau

n = ?

% Vecteur des consignes

w = [?];

**Test du système en boucle fermée :**

> Utilisez la fonction `closedloop` pour tester votre système en boucle fermée (consultez sa description).
>
> Tracez d'une part l'entrée `u` du procédé (calculée par le régulateur), et d'autre part la consigne `w` et la sortie mesurée `y` *sur un même graphique*. 
>
> Vérifiez que la sortie mesurée `y` suit bien les variations de la consigne `w`, avec une certaine dynamique et éventuellement un certain retard. Si ce n'est pas le cas, c'est qu'il y a une erreur dans vos codes et calculs précédentes. Recherchez-la et corrigez-la !

In [ ]:
% À compléter (remplacez les points d'interrogation)
% Simulation du système en boucle fermée

[y,u,t] = closedloop(?);

% Tracé des signaux d'entrée et de sortie

?

## Vérification des performances du régulateur

Plusieurs vérifications doivent être réalisées, et peuvent conduire à réajuster les paramètres de réglage $\omega_0$ et $\zeta$ du régulateur :

1. La commande `u` calculée par le régulateur ne doit pas sortir des limites physiques qui sont ici l'intervalle [0, 100] %. Si les valeurs calculées sont en dehors de cet intervalle, la commande réelle serait "saturée" c'est-à-dire qu'elle resterait à la valeur maximum ou minimum pendant plusieurs périodes d'échantillonnage. Ce comportement est en général à éviter.
2. Le temps d'établissement à 95% en boucle fermée devrait être inférieur à celui du procédé en boucle ouverte (qui vous a été fourni). Autrement dit, le régulateur devrait "accélérer" la réponse.
3. Les dépassements éventuels de consigne devraient rester limités : on accepte généralement un dépassement maximum de 5 à 10% de la variation de la consigne, mais cette valeur dépend des procédés et du contexte.
4. Il ne devrait pas rester d'erreur statique une fois la sortie stabilisée à une valeur de consigne (sans tenir compte du bruit de mesure).


> Procédez à ces vérifications et estimations. 
>
> Pour le temps d'établissement et le dépassement, vous pouvez améliorer la lisibilité des courbes en recréant une consigne plus simple constituée d'un seul créneau (une seule valeur) et en ne traçant que lui et la réponse correspondante du système en boucle fermée. Ces valeurs doivent être estimées *en pourcentage de la variation de la consigne*, et pour le temps d'établissement le retard ne doit pas être pris en compte. Vous pouvez vous contenter d'estimations approximatives lues sur les courbes. 
>
> Pour l'erreur statique, vous devriez pouvoir considérer qu'elle est nulle à l'échelle du graphique (sinon, il y a probablement une erreur dans le calcul du régulateur !).
> 
> Vous pouvez consulter le cours [MCPRO6 - Synthèse des régulateurs RST](https://moodle.bordeaux-inp.fr/mod/resource/view.php?id=22589) pour voir un aperçu de ce que vous devez obtenir et estimer.

In [ ]:
% À compléter (remplacez les points d'interrogation)
% Domaine de variation de la commande u

umin = ?
umax = ?

% Temps d'établissement à 95% en boucle fermée

T95bf = ?

% Dépassement en pourcentage 

d = ?

% Erreur statique

e = ?

Selon les résultats de ces vérifications, il peut être nécessaire de réajuster les paramètres de réglage $\omega_0$ et $\zeta$, recalculer le régulateur, et le remettre en œuvre.

> Vous pouvez relancer facilement toute la procédure avec  d'autres réglages : revenez à la cellule contenant les valeurs de `omega0` et `zeta`, modifiez-les, et dans le menu `Run` choisissez `Run Selected Cell and All Below`.
>
> Même si les vérifications précédentes ne vous semblent pas avoir mis de problèmes en évidence, il est recommandé de faire quelques essais différents pour observer l'effet des paramètres de réglage et faire un choix final.
>
> Une fois que vous êtes satisfait(e) du résultat, c'est terminé !

# Conclusion

> À vous d'écrire cet ultime paragraphe !

# Finalisation et remise de votre travail

- Faites le ménage dans votre Notebook : supprimez toutes les cellules contenant des consignes (celles dont le texte est décalé vers la droite et précédé d'une barre grise verticale). Vous pouvez également supprimer la toute première cellule d'en-tête (celle qui contient "À lire en premier"). Une fois que vous aurez tout vérifié, vous pourrez aussi supprimer ce paragraphe "Finalisation et remise de votre travail" !
- Vérifiez que vous avez bien inscrit vos nom et prénom au début du Notebook.
- Pour vérifier que tout est correct, relancez l'ensemble des calculs depuis le début : dans le menu `Run`, cliquez sur `Run All Cells`. Vérifiez que tous vos résultats s'affichent sans messages d'erreur.
- Enregistrer votre Notebook.
- Dans le menu `File`, choisissez `Export Noteboos As...` puis `Export Noteboos to PDF`. Enregistrez le fichier généré.
    - Note : si vous rencontrez des problèmes pour enregistrer au format PDF, vous pouvez aussi enregistrer votre fichier au format HTML.
- Conseil : ouvrez le fichier généré pour le relire et vérifier que tout vous semble correct. Sinon modifiez votre Notebook et exportez-le à nouveau.
    - Note à propos des fichiers exportés au format PDF : vous pouvez également supprimer la cellule "Sommaire" qui ne fonctionne pas dans ce format.
    - Note à propos des fichiers exportés au format HTML : les figures incluses dans les explications (figures 1, 2, et 3) ne sont pas affichées, n'en tenez pas compte.
- Retournez sur [la page du cours Moodle](https://moodle.bordeaux-inp.fr/course/view.php?id=4&section=5) et déposez votre fichier dans "Remise des comptes-rendus de TP MCP".